# Train model

In [1]:
#from main import run
#run(epochs=3, lr=0.01, batch_size=512)

# Evaluate model

In [2]:
import torch

device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")

/home/mk/anaconda3/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
from dataset import DwgDataset
dwg_dataset = DwgDataset('test_dataset_cluster_labeled.pickle', batch_size = 32, limit_seq_len=1750)

stroke_features = dwg_dataset.entities.stroke_features
max_seq_length = dwg_dataset.entities.max_seq_length

enc_hidden_size    = 256
dec_hidden_size    = 512

d_z=128
n_distributions    = 20

In [4]:
from model import DecoderRNN, EncoderRNN
encoder = EncoderRNN(d_z, enc_hidden_size, stroke_features=stroke_features)
decoder = DecoderRNN(d_z, dec_hidden_size, n_distributions, stroke_features=stroke_features)
encoder.to(device)
decoder.to(device)

# https://pytorch.org/tutorials/beginner/saving_loading_models.html
encoder.load_state_dict(torch.load('DimEncoder.model', map_location=device))
decoder.load_state_dict(torch.load('DimDecoder.model', map_location=device))

decoder.eval()
encoder.eval()

EncoderRNN(
  (lstm): LSTM(5, 256, bidirectional=True)
  (mu_head): Linear(in_features=512, out_features=128, bias=True)
  (sigma_head): Linear(in_features=512, out_features=128, bias=True)
)

In [5]:
import drawSvg as draw
from test_draw  import draw_batch, Sampler

In [6]:
from IPython.display import Image
from IPython.display import clear_output

In [7]:
for i, batch in enumerate(dwg_dataset.train_loader):
    
    data = batch[0].to(device).transpose(0, 1)
    mask = batch[1].to(device).transpose(0, 1)
    
    # draw_batch(data)
    sampler = Sampler(encoder, decoder)
    seq = sampler.sample(data, 0.4)
    print(seq)
    break

tensor([0., 0., 1., 0., 0.]) tensor([[ 1.7423, -1.7068, -0.4441,  ...,  0.4960,  0.2256,  0.6279],
        [-1.4311,  1.2573, -0.6545,  ...,  1.4009,  0.5997, -0.8359],
        [ 1.3905,  1.9572,  0.0296,  ..., -0.0417, -0.7487, -0.5701],
        ...,
        [-1.7434, -2.2072, -1.0875,  ..., -0.4209,  2.0437, -0.2036],
        [ 2.2016, -0.6291,  1.1580,  ..., -2.0095, -0.1660, -0.4531],
        [ 0.2824, -0.6176, -0.4795,  ...,  0.8758,  0.2144,  0.5349]],
       grad_fn=<AddBackward0>)


RuntimeError: Sizes of tensors must match except in dimension 2. Got 1 and 32 in dimension 1 (The offending index is 1)